## Лабораторная работа №1 (Проведение исследований с алгоритмом KNN)

## Классификация

# 1. Выбор начальных условий

**a. Выбор набора данных**

Я выбрал датасет **Credit Risk Dataset**, потому что мне понравилась тема - предсказание кредитного дефолта, что отлично подходит для классификации. В нём около 32к записей, то есть данных достаточно для нормального обучения модели. Табличные финансовые и демографические признаки понятные и удобны для обработки, кодирования и экспериментов с KNN и другими алгоритмами.

**c. Выбор метрик**

- Accuracy - общая доля правильных предсказаний. Показывает, насколько часто модель угадывает правильно. Используем как базовый показатель качества.

- Precision - доля предсказанных дефолтов, которые действительно являются дефолтами. Важна для банка, чтобы не ошибочно классифицировать добросовестных клиентов как рискованных.

- Recall - доля реальных дефолтов, которые модель смогла выявить. Критична для банка, чтобы не пропустить потенциальные убытки.

- F1-Score - гармоническое среднее Precision и Recall. Балансирует оба типа ошибок и особенно полезна, когда классы несбалансированы (дефолтов обычно меньше, чем успешных кредитов).

- ROC-AUC - показывает способность модели различать дефолтные и недефолтные кредиты при разных порогах вероятности. Хороша для сравнения разных моделей и оценки качества ранжирования кредитов по риску.

# 2. Создание бейзлайна и оценка качества

**a. Обучение модели sklearn**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Кодируем категориальные признаки
for col in ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Заполняем пропуски медианой
df = df.fillna(df.median())

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучение KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Предсказания
y_pred = knn.predict(X_test)
y_pred_proba = knn.predict_proba(X_test)[:, 1]

# Метрики
print("KNN (k=5) Бейзлайн")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba):.4f}")

KNN (k=5) Бейзлайн
Accuracy:  0.8716
Precision: 0.7882
Recall:    0.5626
F1-Score:  0.6565
ROC-AUC:   0.8438


**b. Оценка качества модели**

- Accuracy:  0.8716
- Precision: 0.7882
- Recall:    0.5626
- F1-Score:  0.6565
- ROC-AUC:   0.8438

Результаты вполне неплохие, но низкий recall меня смущает, потому что модель пропускает почти половину реальных дефолтов, но хотя бы ROC-AUC порадовал, потому что когда я брал другой датасет, там было всё печально и пришлось переделывать. Работает неплохо, но нужны улучшения.

# 3. Улучшение бейзлайна

**a. Формулировка гипотез**

**Гипотеза 1: Препроцессинг**

Можно удалить явные выбросы по возрасту и стажу (person_age < 100, person_emp_length < 100), заполнить пропуски в числовых колонках медианой и перейти с Label Encoding на One-hot для категориальных признаков (person_home_ownership, loan_intent, loan_grade, cb_person_default_on_file), чтобы модель корректнее работала с категориями.

**Гипотеза 2: Подбор k**

Перебрать разные значения k (3–30) и с помощью кросс-валидации по F1-Score выбрать то, которое даёт лучший результат на обучении.

**Гипотеза 3: Балансировка классов (SMOTE)**

Так как классы несбалансированы (дефолтов меньше), на тренировочной выборке применить SMOTE, чтобы выровнять классы, и заново обучить KNN с найденным оптимальным k.

**Гипотеза 4: Новые признаки**

Добавить признаки income_to_loan (доход / сумма кредита) и age_emp_ratio (возраст / стаж) до One-hot encoding, чтобы дать модели более содержательную информацию о клиенте.

**b. Проверка гипотез**

In [ ]:
# Гипотеза 1: препроцессинг (удаление выбросов + One-hot encoding)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы
print("До удаления выбросов:", len(df))
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]
print("После удаления выбросов:", len(df))

# Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot encoding для категориальных признаков
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

print(f"Количество признаков после One-hot: {X.shape[1]}")

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем KNN с k=5
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Предсказания
y_pred = knn.predict(X_test)
y_pred_proba = knn.predict_proba(X_test)[:, 1]

# Метрики
print("Гипотеза 1: препроцессинг")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba):.4f}")

До удаления выбросов: 32581
После удаления выбросов: 31679
Количество признаков после One-hot: 26
Гипотеза 1: препроцессинг
Accuracy:  0.8995
Precision: 0.8692
Recall:    0.6278
F1-Score:  0.7291
ROC-AUC:   0.8662


**Вывод по гипотезе 1:**

Все метрики улучшились, удаление выбросов помогло, One-hot лучше Label, Recall подрос.

Гипотеза 1 подтверждена, можно её использовать.

In [ ]:
# Гипотеза 2: подбор гиперпараметров (оптимальное k)

from sklearn.model_selection import cross_val_score

# Проверяем разные значения k
k_values = [3, 5, 7, 9, 11, 15, 20, 25, 30]
results = []

print("Проверка разных значений k с кросс-валидацией (5-fold):")

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    # Кросс-валидация по F1-Score
    cv_scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='f1')
    mean_score = cv_scores.mean()
    results.append({'k': k, 'mean_f1': mean_score})
    print(f"k={k:2d}: F1-Score (5-fold CV) = {mean_score:.4f}")

# Находим лучшее k
best_k = max(results, key=lambda x: x['mean_f1'])['k']
best_f1_cv = max(results, key=lambda x: x['mean_f1'])['mean_f1']

print(f"\nЛучшее k = {best_k} (F1-Score = {best_f1_cv:.4f})")

# Обучаем KNN с лучшим k
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(X_train, y_train)

y_pred = knn_best.predict(X_test)
y_pred_proba = knn_best.predict_proba(X_test)[:, 1]

print(f"гипотеза 2: подбор к (k={best_k})")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba):.4f}")

Проверка разных значений k с кросс-валидацией (5-fold):
k= 3: F1-Score (5-fold CV) = 0.7038
k= 5: F1-Score (5-fold CV) = 0.7164
k= 7: F1-Score (5-fold CV) = 0.7200
k= 9: F1-Score (5-fold CV) = 0.7169
k=11: F1-Score (5-fold CV) = 0.7140
k=15: F1-Score (5-fold CV) = 0.7121
k=20: F1-Score (5-fold CV) = 0.6940
k=25: F1-Score (5-fold CV) = 0.6954
k=30: F1-Score (5-fold CV) = 0.6773

Лучшее k = 7 (F1-Score = 0.7200)
гипотеза 2: подбор к (k=7)
Accuracy:  0.9020
Precision: 0.8851
Recall:    0.6264
F1-Score:  0.7336
ROC-AUC:   0.8742


**Вывод по гипотезе 2:**

k=7 лучше, чем k=5, все кроме Recall подросло.

Гипотеза 2 подтверждена, k=7 добавим в улучшенный бейзлайн.

In [ ]:
# Гипотеза 3: балансировка классов (SMOTE)

from imblearn.over_sampling import SMOTE

print("Распределение классов ДО SMOTE:")
print(y_train.value_counts())

# Применяем SMOTE на тренировочных данных
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("\nРаспределение классов ПОСЛЕ SMOTE:")
print(pd.Series(y_train_smote).value_counts())

# Нормализуем данные после SMOTE
scaler = StandardScaler()
X_train_smote = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Обучаем KNN с k=7 (лучший k из гипотезы 2)
knn_smote = KNeighborsClassifier(n_neighbors=7)
knn_smote.fit(X_train_smote, y_train_smote)

y_pred = knn_smote.predict(X_test_scaled)
y_pred_proba = knn_smote.predict_proba(X_test_scaled)[:, 1]

print("гипотеза 3: SMOTE (k=7)")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba):.4f}")

Распределение классов ДО SMOTE:
loan_status
0    19883
1     5460
Name: count, dtype: int64

Распределение классов ПОСЛЕ SMOTE:
loan_status
0    19883
1    19883
Name: count, dtype: int64
гипотеза 3: SMOTE (k=7)
Accuracy:  0.8243
Precision: 0.5684
Recall:    0.7670
F1-Score:  0.6529
ROC-AUC:   0.8735


**Вывод по Гипотезе 3:**

SMOTE помог Recall, но испортил остальное.

Гипотеза 3 не подтверждена, соответсвенно игнорируется.

In [ ]:
# гипотеза 4: новые признаки

df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Создаём новые признаки до One-hot encoding
df['income_to_loan'] = df['person_income'] / (df['loan_amnt'] + 1)  # +1 чтобы избежать деления на 0
df['age_emp_ratio'] = (df['person_age'] + 1) / (df['person_emp_length'] + 1)  # +1 для безопасности

print(f"Новые признаки созданы:")
print(f"  - income_to_loan: min={df['income_to_loan'].min():.2f}, max={df['income_to_loan'].max():.2f}")
print(f"  - age_emp_ratio: min={df['age_emp_ratio'].min():.2f}, max={df['age_emp_ratio'].max():.2f}")

# One-hot encoding для категориальных признаков
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

print(f"\nКоличество признаков (без новых): 26")
print(f"Количество признаков (с новыми): {X.shape[1]}")

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучаем KNN с k=7
knn_fe = KNeighborsClassifier(n_neighbors=7)
knn_fe.fit(X_train, y_train)

y_pred = knn_fe.predict(X_test)
y_pred_proba = knn_fe.predict_proba(X_test)[:, 1]

print("гипотеза 4: новые признаки(k=7)")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba):.4f}")

Новые признаки созданы:
  - income_to_loan: min=1.20, max=1265.82
  - age_emp_ratio: min=1.38, max=74.00

Количество признаков (без новых): 26
Количество признаков (с новыми): 28
гипотеза 4: новые признаки(k=7)
Accuracy:  0.8988
Precision: 0.8943
Recall:    0.6015
F1-Score:  0.7192
ROC-AUC:   0.8680


**Вывод по гипотезе 4:**

Новые признаки особо не улучшили качество, только Precision чутка вырос.


Гипотеза 4 не подтверждена, игнорируется.

**c., d. и e. Формирование, обучение и оценка качества улучшенного бейзлайна**


In [ ]:
# Улучшенный бейзлайн
# гипотеза 1 (Препроцессинг) + гипотеза 2 (k=7)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('credit_risk_dataset.csv')

# Улучшение 1: Удаляем выбросы
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Улучшение 2: Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Улучшение 3: One-hot encoding
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем на признаки и целевой класс
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Улучшение 4: k=7 (вместо k=5)
knn_improved = KNeighborsClassifier(n_neighbors=7)
knn_improved.fit(X_train, y_train)

# Предсказания
y_pred = knn_improved.predict(X_test)
y_pred_proba = knn_improved.predict_proba(X_test)[:, 1]

# Метрики
print("Улучшенный бейзлайн KNN")
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba):.4f}")

Улучшенный бейзлайн KNN
Accuracy:  0.9020
Precision: 0.8851
Recall:    0.6264
F1-Score:  0.7336
ROC-AUC:   0.8742


**f. Сравнение резульатов с пунктом 2**

Базовый KNN:

Accuracy: 0.8716

Precision: 0.7882

Recall: 0.5626

F1-Score: 0.6565

ROC-AUC: 0.8438

Улучшенный KNN:

Accuracy: 0.9020

Precision: 0.8851

Recall: 0.6264

F1-Score: 0.7336

ROC-AUC: 0.8742

Все метрики выросли, что показывает явное улучшение модели. Accuracy увеличилась на 3%, Precision стал выше почти на 10%, Recall подрос с 56% до 62%, а F1-Score улучшился на 8%. Это произошло благодаря улучшенному препроцессингу (удаление выбросов и One-hot encoding) и подбору оптимального k=7 вместо дефолтных k=5.​

**g. Выводы**

Удалось значительно улучшить качество модели по сравнению с базовым бейзлайном. Основной вклад внесли две подтвержденные гипотезы: улучшенный препроцессинг данных (удаление выбросов и переход на One-hot encoding) и подбор оптимального гиперпараметра k=7 через кросс-валидацию. Гипотеза с SMOTE не сработала, потому что хоть Recall и вырос, остальные метрики сильно просели, поэтому её отбросил. Добавление новых признаков тоже особо не помогло. В итоге получил рабочую модель с хорошим балансом между precision и recall.

# 4. Имплементация алгоритма машинного обучения

**a. Имплементация KNN**

In [ ]:
# Имплементация KNN

import numpy as np
from collections import Counter

class KNNClassifier:

    def __init__(self, k=7):
        """
        k: количество соседей для рассмотрения
        """
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        """
        Сохраняем тренировочные данные
        """
        self.X_train = X_train
        self.y_train = y_train

    def euclidean_distance(self, x1, x2):
        """
        Вычисляем евклидово расстояние между двумя точками
        """
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def predict_single(self, x):
        """
        Предсказываем класс для одной точки
        """
        # Вычисляем расстояния до всех тренировочных точек
        distances = []
        for x_train in self.X_train:
            dist = self.euclidean_distance(x, x_train)
            distances.append(dist)

        # Находим индексы k ближайших соседей
        distances = np.array(distances)
        k_indices = np.argsort(distances)[:self.k]

        # Берём классы k ближайших соседей
        k_nearest_labels = self.y_train[k_indices]

        # Возвращаем самый частый класс
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

    def predict(self, X_test):
        """
        Предсказываем классы для всех тестовых точек
        """
        predictions = []
        for x in X_test:
            pred = self.predict_single(x)
            predictions.append(pred)
        return np.array(predictions)

**b. Обучение имплементированной модели с базовым бейзлайном**

In [ ]:
df_base = pd.read_csv('credit_risk_dataset.csv')

# Label Encoding
for col in ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']:
    le = LabelEncoder()
    df_base[col] = le.fit_transform(df_base[col])

# Заполняем пропуски
df_base = df_base.fillna(df_base.median())

# Разделяем
X_base = df_base.drop('loan_status', axis=1)
y_base = df_base['loan_status']

X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, test_size=0.2, random_state=42, stratify=y_base)

# Нормализуем
scaler_base = StandardScaler()
X_train_base = scaler_base.fit_transform(X_train_base)
X_test_base = scaler_base.transform(X_test_base)

# Конвертируем в numpy
X_train_base = np.array(X_train_base)
X_test_base = np.array(X_test_base)
y_train_base = np.array(y_train_base)
y_test_base = np.array(y_test_base)

# Используем KNNClassifier
knn_base = KNNClassifier(k=5)
knn_base.fit(X_train_base, y_train_base)

print("KNN без улучшений обучен")

y_pred_base = knn_base.predict(X_test_base)

KNN без улучшений обучен


**c. Оценка качества имплементированной модели с базовым бейзлайном**

In [ ]:
# Вычисляем вероятности для ROC-AUC
y_pred_proba_base = []
for i, x_test in enumerate(X_test_base):
    distances = np.sqrt(np.sum((X_train_base - x_test) ** 2, axis=1))
    k_indices = np.argsort(distances)[:5]
    k_nearest_labels = y_train_base[k_indices]
    prob_class_1 = np.sum(k_nearest_labels) / 5
    y_pred_proba_base.append(prob_class_1)
    if (i + 1) % 500 == 0:
        print(f"Обработано {i + 1}/{len(X_test_base)}")

y_pred_proba_base = np.array(y_pred_proba_base)

print("Имплементированный KNN без улучшений (k=5)")
print(f"Accuracy:  {accuracy_score(y_test_base, y_pred_base):.4f}")
print(f"Precision: {precision_score(y_test_base, y_pred_base):.4f}")
print(f"Recall:    {recall_score(y_test_base, y_pred_base):.4f}")
print(f"F1-Score:  {f1_score(y_test_base, y_pred_base):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test_base, y_pred_proba_base):.4f}")

Обработано 500/6517
Обработано 1000/6517
Обработано 1500/6517
Обработано 2000/6517
Обработано 2500/6517
Обработано 3000/6517
Обработано 3500/6517
Обработано 4000/6517
Обработано 4500/6517
Обработано 5000/6517
Обработано 5500/6517
Обработано 6000/6517
Обработано 6500/6517
Имплементированный KNN без улучшений (k=5)
Accuracy:  0.8716
Precision: 0.7882
Recall:    0.5626
F1-Score:  0.6565
ROC-AUC:   0.8438


**d. Сравнение результатов с пунктом 2**

Результаты полностью совпали, имплементированный KNN дает точно такие же зачения метрик, как и sklearn.
- Accuracy:  0.8716
- Precision: 0.7882
- Recall:    0.5626
- F1-Score:  0.6565
- ROC-AUC:   0.8438

**e. Выводы**

Результаты сошлись со вторым пунктом, это говорит о правильности, но время выполнения было заметно больше, чем в случае sklearn.

**f. и g. Добавление техник из улучшенного бейзлайна и обучение**

In [ ]:
# Имплементация с улучшеннным бейзлайном

df = pd.read_csv('credit_risk_dataset.csv')

# Удаляем выбросы
df = df[(df['person_age'] < 100) & (df['person_emp_length'] < 100)]

# Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# One-hot encoding
df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file'], drop_first=False)

# Разделяем
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Нормализуем
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# конвертируем в numpy
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Обучаем
knn_custom = KNNClassifier(k=7)
knn_custom.fit(X_train, y_train)

print("KNN обучен")

# Предсказания
y_pred_custom = knn_custom.predict(X_test)

print(f"Размер тестового набора: {len(X_test)}")
print(f"Первые 10 предсказаний: {y_pred_custom[:10]}")

KNN обучен
Размер тестового набора: 6336
Первые 10 предсказаний: [0 0 0 0 0 0 0 1 0 0]


**h. Оценка качества имплементированной модели с улучшенным бейзлайном**



In [ ]:
y_pred_proba_custom = []
for i, x_test in enumerate(X_test):
    distances = np.sqrt(np.sum((X_train - x_test) ** 2, axis=1))
    k_indices = np.argsort(distances)[:7]
    k_nearest_labels = y_train[k_indices]
    prob_class_1 = np.sum(k_nearest_labels) / 7  # Доля класса 1 среди соседей
    y_pred_proba_custom.append(prob_class_1)
    if (i + 1) % 500 == 0:
        print(f"Обработано {i + 1}/{len(X_test)}")

y_pred_proba_custom = np.array(y_pred_proba_custom)

print("имплементированный KNN (k=7) с улучшениями")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_custom):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_custom):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_custom):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred_custom):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba_custom):.4f}")

Обработано 500/6336
Обработано 1000/6336
Обработано 1500/6336
Обработано 2000/6336
Обработано 2500/6336
Обработано 3000/6336
Обработано 3500/6336
Обработано 4000/6336
Обработано 4500/6336
Обработано 5000/6336
Обработано 5500/6336
Обработано 6000/6336
имплементированный KNN (k=7) с улучшениями
Accuracy:  0.9020
Precision: 0.8851
Recall:    0.6264
F1-Score:  0.7336
ROC-AUC:   0.8742


**i. Сравнение с результатами из пункта 3**

Результаты полностью сошлись с третьим пунктом
- Accuracy:  0.9020
- Precision: 0.8851
- Recall:    0.6264
- F1-Score:  0.7336
- ROC-AUC:   0.8742

**j. Выводы**

Понадобилось намного больше времени на ожидание результатов по сравнению с KNN sklearn, но тем не менее цифры получились те же.

## Регрессия

# 1. Выбор начальных условий

**b. Выбор набора данных**

Я выбрал датасет **India Job Market & Salary Trends**, потому что на его основе можно предсказывать зарплаты по таким параметрам, как должность, опыт, город, компания, формат работы и уровень спроса. Можно понять, какие зарплаты получают разные IT-специалисты в разных городах, а компаниям оценить конкуренцию и уровень рынка.


**c. Выбор метрик и их обоснование**

- MAE (Mean Absolute Error)
Средняя ошибка в рупиях. Если MAE = 50000, значит модель в среднем промахивается на 50 тысяч рупий.

- RMSE (Root Mean Squared Error)
Тоже средняя ошибка в рупиях, но сильнее штрафует большие промахи. Если модель иногда ошибается на миллион рупий, RMSE это покажет лучше чем MAE

- R² (коэффициент детерминации)
Показывает, насколько хорошо модель объясняет данные. Значение от 0 до 1, чем ближе к 1, тем лучше

- MAPE (Mean Absolute Percentage Error)
Средняя ошибка в процентах. Если MAPE = 10%, значит модель в среднем промахивается на 10% от реальной зарплаты. Удобно для сравнения точности на разных уровнях зарплат

Эти метрики смотрят на качество модели с разных сторон: абсолютные ошибки в деньгах, общее качество предсказания и относительная точность. Вместе они дают полную картину того, как работает модель на данных о зарплатах.

# 2. Создание бейзлайна и оценка качества

**a. Обучение модели sklearn**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


df = pd.read_csv('Job_Market_India.csv')


# Убираем ненужные колонки
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Кодируем категориальные признаки
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


# Заполняем пропуски медианой
df = df.fillna(df.median())


# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Обучение
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)


# Предсказания
y_pred = knn.predict(X_test)


# Вычисляем метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100


# Вывод метрик
print("KNN (k=5) Бейзлайн")
print(f"MAE:   {mae:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"R²:    {r2:.4f}")
print(f"MAPE:  {mape:.2f}%")

KNN (k=5) Бейзлайн
MAE:   580637.80
RMSE:  769368.94
R²:    0.0597
MAPE:  58.33%


**b. Оценка качества модели**

Модель показывает слабоватые результаты на базовом бейзлайне. R² равен всего 0.0597, то есть модель объясняет только 6% разброса зарплат, что очень мало. MAPE составляет 58%, то есть в среднем модель ошибается больше чем на половину реальной зарплаты. MAE около 580 тысяч рупий и RMSE около 769 тысяч показывают большую абсолютную ошибку в предсказаниях. Такое низкое качество я могу объяснить тем, что это базовый бейзлайн без препроцессинга: используется простой Label Encoding для категориальных признаков, не удалены выбросы, не подобраны гиперпараметры.

# 3. Улучшение бейзлайна

**a. Формулировка гипотез**

**Гипотеза 1: Препроцессинг**

Удалить выбросы в зарплатах (слишком высокие или низкие значения искажают модель)

Использовать One-hot encoding вместо Label Encoding для категориальных признаков (Company_Name, Job_Role, City, Experience_Level)

**Гипотеза 2: Подбор гиперпараметров**

Найти оптимальное k (вместо k=5) с помощью кросс-валидации

Проверить разные значения k от 3 до 30

**Гипотеза 3: Новые признаки**

Создать новый признаки: среднюю зарплату по городу, среднюю зарплату по должности, соотношение Demand_Index к Remote_Option_Flag

**Гипотеза 4: Масштабирование целевой переменной**

Попробовать логарифмирование зарплаты для уменьшения разброса значений

Проверить, улучшит ли это способность модели предсказывать на разных уровнях зарплат

**b. Проверка гипотез**

In [ ]:
# Гипотеза 1: препроцессинг (удаление выбросов + One-hot encoding)


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


df = pd.read_csv('Job_Market_India.csv')


# Убираем ненужные колонки
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Удаляем выбросы в зарплате
print("До удаления выбросов:", len(df))
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]
print("После удаления выбросов:", len(df))


# Заполняем пропуски только в числовых колонках
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# One-hot encoding для категориальных признаков
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)


# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


print(f"Количество признаков после One-hot: {X.shape[1]}")


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Обучаем KNN с k=5
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)


# Предсказания
y_pred = knn.predict(X_test)


# Вычисляем метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100


# Метрики
print("Гипотеза 1: препроцессинг")
print(f"MAE:   {mae:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"R²:    {r2:.4f}")
print(f"MAPE:  {mape:.2f}%")

До удаления выбросов: 30000
После удаления выбросов: 29400
Количество признаков после One-hot: 66
Гипотеза 1: препроцессинг
MAE:   416551.74
RMSE:  537939.58
R²:    0.4488
MAPE:  41.15%


**Вывод по гипотезе 1:**

R² вырос с 6% до 45%, то есть модель теперь объясняет почти половину разброса зарплат. MAPE снизился с 58% до 41%, ошибки стали меньше. Удаление выбросов и One-hot encoding значительно улучшили качество.

Гипотеза 1 подтверждена, используем.

In [ ]:
# Гипотеза 2: подбор гиперпараметров (оптимальное k)


from sklearn.model_selection import cross_val_score


# Проверяем разные значения k
k_values = [3, 5, 7, 9, 11, 15, 20, 25, 30]
results = []


print("Проверка разных значений k с кросс-валидацией (5-fold):")


for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    # Кросс-валидация по R²
    cv_scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='r2')
    mean_score = cv_scores.mean()
    results.append({'k': k, 'mean_r2': mean_score})
    print(f"k={k:2d}: R² (5-fold CV) = {mean_score:.4f}")


# Находим лучшее k
best_k = max(results, key=lambda x: x['mean_r2'])['k']
best_r2_cv = max(results, key=lambda x: x['mean_r2'])['mean_r2']


print(f"\nЛучшее k = {best_k} (R² = {best_r2_cv:.4f})")


# Обучаем KNN с лучшим k
knn_best = KNeighborsRegressor(n_neighbors=best_k)
knn_best.fit(X_train, y_train)


y_pred = knn_best.predict(X_test)


# Вычисляем метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100


print(f"Гипотеза 2: подбор k (k={best_k})")
print(f"MAE:   {mae:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"R²:    {r2:.4f}")
print(f"MAPE:  {mape:.2f}%")

Проверка разных значений k с кросс-валидацией (5-fold):
k= 3: R² (5-fold CV) = 0.3989
k= 5: R² (5-fold CV) = 0.4564
k= 7: R² (5-fold CV) = 0.4769
k= 9: R² (5-fold CV) = 0.4833
k=11: R² (5-fold CV) = 0.4851
k=15: R² (5-fold CV) = 0.4861
k=20: R² (5-fold CV) = 0.4889
k=25: R² (5-fold CV) = 0.4911
k=30: R² (5-fold CV) = 0.4930

Лучшее k = 30 (R² = 0.4930)
Гипотеза 2: подбор k (k=30)
MAE:   407087.34
RMSE:  516451.79
R²:    0.4919
MAPE:  41.47%


**Вывод по гипотезе 2:**

R² вырос с 45% до 49%, модель стала лучше объяснять разброс зарплат. MAE и RMSE снизились, значит ошибки меньше. MAPE чуть вырос, но это некритично. Кросс-валидация подтвердила, что k=30 оптимальнее чем k=5.

Гипотеза 2 подтвердилась, используем

In [ ]:
# Гипотеза 3: Новые признаки

df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Удаляем выбросы
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]


# Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# Создаем новые признаки до One-hot encoding
df['avg_salary_by_city'] = df.groupby('City')['Salary_INR'].transform('mean')
df['avg_salary_by_role'] = df.groupby('Job_Role')['Salary_INR'].transform('mean')
df['demand_remote_ratio'] = df['Demand_Index'] / (df['Remote_Option_Flag'] + 1)


print(f"Новые признаки созданы:")
print(f"  - avg_salary_by_city")
print(f"  - avg_salary_by_role")
print(f"  - demand_remote_ratio")


# One-hot encoding
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)


# Разделяем
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


print(f"\nКоличество признаков (без новых): 66")
print(f"Количество признаков (с новыми): {X.shape[1]}")


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Обучаем KNN с k=30
knn_fe = KNeighborsRegressor(n_neighbors=30)
knn_fe.fit(X_train, y_train)


y_pred = knn_fe.predict(X_test)


# Метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100


print("Гипотеза 3: Новые признаки (k=30)")
print(f"MAE:   {mae:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"R²:    {r2:.4f}")
print(f"MAPE:  {mape:.2f}%")

Новые признаки созданы:
  - avg_salary_by_city
  - avg_salary_by_role
  - demand_remote_ratio

Количество признаков (без новых): 66
Количество признаков (с новыми): 69
Гипотеза 3: Новые признаки (k=30)
MAE:   402005.45
RMSE:  506240.34
R²:    0.5118
MAPE:  40.47%


**Вывод по гипотезе 3:**

R² вырос с 49% до 51%, модель теперь объясняет больше половины разброса зарплат. Все ошибки снизились.

Гипотеза 3 подтверждена, используем.

In [ ]:
# Гипотеза 4: масштабирование целевой переменной


# Загружаем данные заново
df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Удаляем выбросы
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]


# Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# Логарифмируем целевую переменную
df['Salary_INR_log'] = np.log1p(df['Salary_INR'])


print("Масштабирование целевой переменной:")
print(f"  До: min={df['Salary_INR'].min():.0f}, max={df['Salary_INR'].max():.0f}")
print(f"  После log: min={df['Salary_INR_log'].min():.2f}, max={df['Salary_INR_log'].max():.2f}")


# One-hot encoding
df_encoded = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)


# Разделяем (используем логарифмированную зарплату)
X = df_encoded.drop(['Salary_INR', 'Salary_INR_log'], axis=1)
y_log = df_encoded['Salary_INR_log']
y_original = df_encoded['Salary_INR']


# Train/test разделение
X_train, X_test, y_train_log, y_test_log, y_train_orig, y_test_orig = train_test_split(
    X, y_log, y_original, test_size=0.2, random_state=42
)


# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Обучаем KNN с k=30 на логарифмированных данных
knn_log = KNeighborsRegressor(n_neighbors=30)
knn_log.fit(X_train, y_train_log)


# Предсказания в логарифмической шкале
y_pred_log = knn_log.predict(X_test)


# Обратное преобразование в исходную шкалу
y_pred = np.expm1(y_pred_log)


# Метрики на исходной шкале
mae = mean_absolute_error(y_test_orig, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred))
r2 = r2_score(y_test_orig, y_pred)
mape = np.mean(np.abs((y_test_orig - y_pred) / y_test_orig)) * 100


print("\nГипотеза 4: логарифмирование целевой переменной (k=30)")
print(f"MAE:   {mae:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"R²:    {r2:.4f}")
print(f"MAPE:  {mape:.2f}%")

Масштабирование целевой переменной:
  До: min=338096, max=3974203
  После log: min=12.73, max=15.20

Гипотеза 4: логарифмирование целевой переменной (k=30)
MAE:   413992.04
RMSE:  542083.63
R²:    0.4403
MAPE:  37.85%


**Вывод по гипотезе 4:**

R² упал с 49% до 44%, MAE и RMSE выросли. MAPE немного улучшился, но это не компенсирует ухудшение остальных метрик. Логарифмирование не помогло для этого датасета.

Гипотеза 4 не подтверждена, не используем.

**c. и d. Формирование улучшенного бейзлайна и обучение модели**

In [ ]:
# Гипотеза 1 (Препроцессинг) + гипотеза 2 (k=30) + гипотеза 3 (новые признаки)


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Улучшение 1: Удаляем выбросы
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]


# Улучшение 2: Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# Улучшение 3: Создаем новые признаки
df['avg_salary_by_city'] = df.groupby('City')['Salary_INR'].transform('mean')
df['avg_salary_by_role'] = df.groupby('Job_Role')['Salary_INR'].transform('mean')
df['demand_remote_ratio'] = df['Demand_Index'] / (df['Remote_Option_Flag'] + 1)


# Улучшение 4: One-hot encoding
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)


# Разделяем на признаки и целевую переменную
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


# Train/test разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Нормализуем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Улучшение 5: k=30 (вместо k=5)
knn_improved = KNeighborsRegressor(n_neighbors=30)
knn_improved.fit(X_train, y_train)


# Предсказания
y_pred = knn_improved.predict(X_test)


# Метрики
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100


print("Улучшенный бейзлайн KNN")
print(f"MAE:   {mae:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"R²:    {r2:.4f}")
print(f"MAPE:  {mape:.2f}%")

Улучшенный бейзлайн KNN
MAE:   402005.45
RMSE:  506240.34
R²:    0.5118
MAPE:  40.47%


**f. Сравнение результатов с пунктом 2**

**Обычный бейзлайн:**

MAE  = 580637.80

RMSE = 769368.94

R²   = 0.0597

MAPE = 58.33%



**Улучшенный бейзлайн:**

MAE  = 402005.45

RMSE = 506240.34

R²   = 0.5118

MAPE = 40.47%

Видно, что после улучшений модель стала гораздо лучше объяснять данные (R² с 6% до 51%) и заметно снизила как абсолютные, так и относительные ошибки в предсказании зарплат.

**g. Выводы**

Все метрики значительно улучшились. R² вырос с 6% до 51%, то есть модель теперь объясняет больше половины разброса зарплат вместо почти ничего. MAE снизилась на 178 тысяч рупий, MAPE упал с 58% до 40%. Улучшения достигнуты за счет удаления выбросов, One-hot encoding, создания новых признаков и подбора оптимального k=30.

# 4. Имплементация алгоритма машинного обучения

**a. Имплементация**

In [ ]:
import numpy as np
from collections import Counter


class KNNRegressor:

    def __init__(self, k=5):
        """
        k: количество соседей для рассмотрения
        """
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        """
        Сохраняем тренировочные данные
        """
        self.X_train = X_train
        self.y_train = y_train

    def euclidean_distance(self, x1, x2):
        """
        Вычисляем евклидово расстояние между двумя точками
        """
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def predict_single(self, x):
        """
        Предсказываем значение для одной точки
        """
        # Вычисляем расстояния до всех тренировочных точек
        distances = []
        for x_train in self.X_train:
            dist = self.euclidean_distance(x, x_train)
            distances.append(dist)

        # Находим индексы k ближайших соседей
        distances = np.array(distances)
        k_indices = np.argsort(distances)[:self.k]

        # Берём значения k ближайших соседей
        k_nearest_values = self.y_train[k_indices]

        # Возвращаем среднее значение
        return np.mean(k_nearest_values)

    def predict(self, X_test):
        """
        Предсказываем значения для всех тестовых точек
        """
        predictions = []
        for x in X_test:
            pred = self.predict_single(x)
            predictions.append(pred)
        return np.array(predictions)


print("KNN имплементирован")

KNN имплементирован


**b. Обучение имплементированной модели с базовым бейзлайном**

In [ ]:
# Обучаем KNN на данных без улучшений


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Label Encoding
for col in ['Company_Name', 'Job_Role', 'Experience_Level', 'City']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


# Заполняем пропуски
df = df.fillna(df.median())


# Разделяем
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Нормализуем
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Конвертируем в numpy
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


# Используем KNNRegressor (k=5)
knn_base = KNNRegressor(k=5)
knn_base.fit(X_train, y_train)


print("KNN без улучшений обучен")


y_pred_base = knn_base.predict(X_test)


print(f"Размер тестового набора: {len(X_test)}")
print(f"Первые 10 предсказаний: {y_pred_base[:10]}")

KNN без улучшений обучен
Размер тестового набора: 6000
Первые 10 предсказаний: [1031784.2 1142354.2  654170.2 1761596.4 1412841.   760500.  1623495.6
  924672.  1024848.4 1192255.4]


**c. Оценка качества имплементированной модели с базовым бейзлайном**



In [ ]:
# Оценка качества имплементированного KNN без улучшений


# Вычисляем метрики
mae = mean_absolute_error(y_test, y_pred_base)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_base))
r2 = r2_score(y_test, y_pred_base)
mape = np.mean(np.abs((y_test - y_pred_base) / y_test)) * 100


print("Имплементированный KNN с простым бейзлайном (k=5)")
print(f"MAE:   {mae:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"R²:    {r2:.4f}")
print(f"MAPE:  {mape:.2f}%")

Имплементированный KNN с простым бейзлайном (k=5)
MAE:   580607.22
RMSE:  769253.75
R²:    0.0600
MAPE:  58.33%


**d. Сравнение результатов модели с результатами из пункта 2**

**Имплементированный:**

MAE:   580607.22

RMSE:  769253.75

R²:    0.0600

MAPE:  58.33%

**Sklearn:**

MAE:   580637.80

RMSE:  769368.94

R²:    0.0597

MAPE:  58.33%

Результаты совпадают на 99%, минимальные различия из-за округлений при вычислениях.

**e. Выводы**

Имплементированный KNN без улучшений дает результаты, идентичные sklearn KNN на 99.9%. Все метрики совпадают: MAE около 580 тысяч рупий, RMSE около 769 тысяч, R² = 6%, MAPE = 58%. Это доказывает корректность реализации алгоритма, вычисление евклидовых расстояний, поиск k ближайших соседей и усреднение их значений работают правильно.

**f. и g. Добавление техник из улучшенного бейзлайна и обучение модели**

In [ ]:
df = pd.read_csv('Job_Market_India.csv')
df = df.drop(['Record_Date', 'Salary_Trend_Pct'], axis=1)


# Улучшение 1: Удаляем выбросы
q1 = df['Salary_INR'].quantile(0.01)
q99 = df['Salary_INR'].quantile(0.99)
df = df[(df['Salary_INR'] >= q1) & (df['Salary_INR'] <= q99)]


# Улучшение 2: Заполняем пропуски
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# Улучшение 3: Создаем новые признаки
df['avg_salary_by_city'] = df.groupby('City')['Salary_INR'].transform('mean')
df['avg_salary_by_role'] = df.groupby('Job_Role')['Salary_INR'].transform('mean')
df['demand_remote_ratio'] = df['Demand_Index'] / (df['Remote_Option_Flag'] + 1)


# Улучшение 4: One-hot encoding
df = pd.get_dummies(df, columns=['Company_Name', 'Job_Role', 'Experience_Level', 'City'], drop_first=False)


# Разделяем
X = df.drop('Salary_INR', axis=1)
y = df['Salary_INR']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Нормализуем
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Конвертируем в numpy
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


# Используем KNNRegressor с k=30
knn_improved = KNNRegressor(k=30)
knn_improved.fit(X_train, y_train)


print("KNN С улучшениями обучен")


y_pred_improved = knn_improved.predict(X_test)


print(f"Размер тестового набора: {len(X_test)}")
print(f"Первые 10 предсказаний: {y_pred_improved[:10]}")

KNN С улучшениями обучен
Размер тестового набора: 5880
Первые 10 предсказаний: [ 981578.66666667  856133.46666667 1323841.2         957188.86666667
 2248575.96666667 1184405.66666667 2369860.23333333 2211527.7
  952631.53333333 1333867.7       ]


**h. Оценка качества имплементированного KNN с улучшенным бейзлайном**

In [ ]:
# Вычисляем метрики
mae = mean_absolute_error(y_test, y_pred_improved)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_improved))
r2 = r2_score(y_test, y_pred_improved)
mape = np.mean(np.abs((y_test - y_pred_improved) / y_test)) * 100


print("Имплементированный KNN С улучшениями (k=30)")
print(f"MAE:   {mae:.2f}")
print(f"RMSE:  {rmse:.2f}")
print(f"R²:    {r2:.4f}")
print(f"MAPE:  {mape:.2f}%")

Имплементированный KNN С улучшениями (k=30)
MAE:   402012.62
RMSE:  506247.17
R²:    0.5118
MAPE:  40.47%


**i. Сравнение результатов с пунктом 3**

**Имплементированный:**

MAE:   402012.62

RMSE:  506247.17

R²:    0.5118

MAPE:  40.47%

**Sklearn:**

MAE = 402005.45

RMSE = 506240.34

R² = 0.5118

MAPE = 40.47%

Результаты почти идентичны.

j. **Выводы**

Результаты моего KNN полностью совпали с результатами KNN из sklearn с улучшенным бейзлайом. Разница в MAE и RMSE меньше 10 рупий при общей сумме в 400к, то есть это просто округление. Значения R² и MAPE одинаковые. Это показывает, что имплементированный KNN работает правильно.